# Capstone Project - The Battle of the Neighborhoods

## 1. Introduction

Toronto is the provincial capital of Ontario. With a recorded population of 2,731,571 in 2016, it is the most populous city in Canada and the fourth most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,245,438 people (as of 2016) surrounding the western end of Lake Ontario, while the Greater Toronto Area (GTA) proper had a 2016 population of 6,417,516. Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world.

New York City, often called simply New York and abbreviated as NYC, is the most populous city in the United States. With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles (784 km2), New York City is also the most densely populated major city in the United States. Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass. With almost 20 million people in its metropolitan statistical area and approximately 23 million in its combined statistical area, it is one of the world's most populous megacities. New York City has been described as the cultural, financial, and media capital of the world, significantly influencing commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports. Home to the headquarters of the United Nations, New York is an important center for international diplomacy.

#### The problem
Now let me explain the context of this Capstone project through a scenario. Suppose a friend who live on the west side of the city of Toronto in Canada, receive a job offer from a great company in New York, Manhattan borough. He has to move to New York City. He love his neighborhood in Toronto beacause of its variety of venues for food, parks, schools and entertainment places. Consequently he want to move in a similar zone in Manhattan borough. The aim of this project is to study and analyze the neighborhoods of Toronto city and New York city and group them into similar clusters. Finally I will use those information to find the most similar neighborhood of the two borough of the two cities.

## 2. The Data

For this project we need the following data :

1. New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.
 - Data source : https://cocl.us/new_york_dataset
2. Toronto data that contains list Boroughs, Neighborhoods along with their latitude and longitude
 - Data source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M , http://cocl.us/Geospatial_data
                  

## 3. Methodology

1. Get the data of both cities into pandas dataframe and process them
2. Use the Foursquare API to find all venues for each neighborhood and analyze them
3. Use cluster algorithm to find similar neighborhood
4. Compare clusters between the two cities

## 4. Analysis

### 4.1 Toronto data

Firstly let's import some useful libraries

In [6]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Now let's scrap the data from Wikipedia page into a dataframe

In [7]:
df_tor= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_tor.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df_tor.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The dataframe consist of three columns: PostalCode, Borough, and Neighborhood. Let's drop cells with a borough that is "Not assigned". We notice that more than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. Moreover if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [8]:
df_tor1=df_tor[~df_tor.Borough.str.contains("Not assigned")].reset_index(drop=True)
df_tor1=df_tor1.groupby(['Postal Code','Borough'],as_index=False).agg(lambda x: ", ".join(x))
for index, row in df_tor1.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_tor1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


We also fetched the coordinate data for all the neighborhoods in Toronto using the csv file and put it into a dataframe. Next, we combine both the dataframes i.e. adding the coordinate data to the original dataframe.

In [9]:
df2 = pd.read_csv("Geospatial_Coordinates.csv")
df_tor2=pd.merge(left=df_tor1, right=df2 ,on='Postal Code')
print(df_tor2.shape)
df_tor2.head()

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Now, we filter only boroughs that contain the word Toronto

In [10]:
print(df_tor2.Borough.unique())

['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' 'Mississauga' 'Etobicoke']


In [11]:
borough_with_toronto=['East Toronto','Central Toronto','Downtown Toronto', 'West Toronto']
df_tor3 = df_tor2[df_tor2['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_tor3.shape)
df_tor3.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


We use geopy library to get the latitude and longitude values of Toronto. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer, as shown below.

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


We then use the python folium library to visualize geographic details of Toronto and its boroughs.

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor3['Latitude'], df_tor3['Longitude'], df_tor3['Borough'], df_tor3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### 4.2 New York data

Firstly, let's load the data nedeed

In [14]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [15]:
neighborhoods_data = newyork_data['features']

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [16]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [17]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [18]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Now, for our purpose, let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [19]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [20]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


Let's visualize Manhattan the neighborhoods in it.

In [21]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## 4.3 Toronto's areas analysis 

We are going to start utilizing the Foursquare API to explore the neighborhoods and segment them. Firstly we define Foursquare Credentials and Version. Secondly let's get the top 50 venues that are in the selected Toronto borough s within a radius of 500 meters.

In [22]:
CLIENT_ID = 'QJB2JS4JPRGFMR1NB0PLBJ2LCBSPMVK40TWCI5XTWI1ZXOJE' # your Foursquare ID
CLIENT_SECRET = '5EH3OXJI0WRHMLHFSCUWXLGBEBRSZ4H0XXKZWOEU0CWMSCMT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJB2JS4JPRGFMR1NB0PLBJ2LCBSPMVK40TWCI5XTWI1ZXOJE
CLIENT_SECRET:5EH3OXJI0WRHMLHFSCUWXLGBEBRSZ4H0XXKZWOEU0CWMSCMT


In [36]:
radius = 500
LIMIT = 50

venues = []

for lat, long, post, borough, neighborhood in zip(df_tor3['Latitude'], df_tor3['Longitude'], df_tor3['Postal Code'], df_tor3['Borough'], df_tor3['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [37]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1196, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


We use One Hot Encoding, use the neighborhood to group data, and find out the top ten venues present in each neighborhood.

In [104]:
# one hot encoding
Toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] 
for i in range(0,Toronto_onehot.shape[1]-1):
    fixed_columns.append(Toronto_onehot.columns[i])
Toronto_onehot = Toronto_onehot[fixed_columns]

# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

# Now let's create the new dataframe and display the top 10 venues for each PostalCode
num_top_venues = 10

# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cheese Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Restaurant,Café,Pharmacy
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Bakery,Coffee Shop,Breakfast Spot,Office,Pet Store,Convenience Store,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Pizza Place,Farmers Market,Auto Workshop,Spa,Burrito Place,Restaurant,Gym / Fitness Center,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Harbor / Marina,Plane,Sculpture Garden,Rental Car Location,Airport Gate
4,Central Bay Street,Coffee Shop,Sandwich Place,Bubble Tea Shop,Café,Italian Restaurant,Comic Shop,Salad Place,Burger Joint,Poke Place,Pizza Place


### 4.4 Manhattan areas analysis

Similarly, let's get the top 50 venues that are in the New York Manhattan borough within a radius of 500 meters

In [35]:
radius = 500
LIMIT = 50

venues1 = []

for lat, long, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    
    results = requests.get(url,'none').json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues1.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [41]:
# convert the venues list into a new DataFrame
venues_df1 = pd.DataFrame(venues1)

# define the column names
venues_df1.columns = ['Postal Code','Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df1.shape)
venues_df1.head()

(1887, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M7Y,Manhattan,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,M7Y,Manhattan,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,M7Y,Manhattan,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,M7Y,Manhattan,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,M7Y,Manhattan,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


We use One Hot Encoding, use the neighborhood to group data, and find out the top ten venues present in each neighborhood

In [105]:
# one hot encoding
Manhattan_onehot = pd.get_dummies(venues_df1[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Manhattan_onehot['Neighborhood'] = venues_df1['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Manhattan_onehot.columns[-1]] 
for i in range(0,Manhattan_onehot.shape[1]-1):
    fixed_columns.append(Manhattan_onehot.columns[i])
Manhattan_onehot = Manhattan_onehot[fixed_columns]

# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Manhattan_grouped = Manhattan_onehot.groupby('Neighborhood').mean().reset_index()

# Now let's create the new dataframe and display the top 10 venues for each PostalCode
num_top_venues = 10

# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Manhattan_venues_sorted = pd.DataFrame(columns=columns)
Manhattan_venues_sorted['Neighborhood'] = Manhattan_grouped['Neighborhood']

for ind in np.arange(Manhattan_grouped.shape[0]):
    Manhattan_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manhattan_grouped.iloc[ind, :], num_top_venues)

Manhattan_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Memorial Site,Gourmet Shop,Food Court,Plaza,Hotel,Gym,Shopping Mall,Boat or Ferry
1,Carnegie Hill,Pizza Place,Gym / Fitness Center,Coffee Shop,Bookstore,Bakery,Gym,Italian Restaurant,French Restaurant,Café,Yoga Studio
2,Central Harlem,African Restaurant,Pizza Place,Seafood Restaurant,Cosmetics Shop,American Restaurant,Bar,Fried Chicken Joint,French Restaurant,Chinese Restaurant,Bookstore
3,Chelsea,Coffee Shop,American Restaurant,Italian Restaurant,Seafood Restaurant,Hotel,Ice Cream Shop,French Restaurant,Cupcake Shop,Middle Eastern Restaurant,Liquor Store
4,Chinatown,Chinese Restaurant,American Restaurant,Ice Cream Shop,Boutique,Greek Restaurant,Bubble Tea Shop,Salon / Barbershop,Sandwich Place,Spa,Asian Restaurant


## 4.5 Cluster Neighborhood

Let's aggregate the two dataframe in a singol one

In [106]:
neigh_df=pd.merge(Toronto_venues_sorted, Manhattan_venues_sorted, how='outer')
print(Toronto_venues_sorted1.shape, Manhattan_venues_sorted1.shape, neigh_df.shape)
neigh_df.head()

(39, 11) (40, 11) (79, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cheese Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Restaurant,Café,Pharmacy
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Bakery,Coffee Shop,Breakfast Spot,Office,Pet Store,Convenience Store,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Pizza Place,Farmers Market,Auto Workshop,Spa,Burrito Place,Restaurant,Gym / Fitness Center,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Harbor / Marina,Plane,Sculpture Garden,Rental Car Location,Airport Gate
4,Central Bay Street,Coffee Shop,Sandwich Place,Bubble Tea Shop,Café,Italian Restaurant,Comic Shop,Salad Place,Burger Joint,Poke Place,Pizza Place


We have some common venue categories in the neighborhoods. We use the unsupervised learning K-means algorithm to cluster the neighborhoods. K-Means algorithm is one of the most common method for clustering in unsupervised learning. Let's run k-means to cluster the neighborhood into 5 clusters.

In [107]:
#aggregating the two dataframe
neigh_grouped=pd.merge(Toronto_grouped, Manhattan_grouped, how='outer').reset_index(drop=True)
neigh_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Argentinian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Baseball Field,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boxing Gym,Bridal Shop,Bridge,Building,Bus Station,Cafeteria,Caucasian Restaurant,Circus,Club House,College Academic Building,College Bookstore,College Theater,Comedy Club,Community Center,Cooking School,Cycle Studio,Czech Restaurant,Daycare,Design Studio,Doctor's Office,Drugstore,Dry Cleaner,Duty-free Shop,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Exhibit,Food Stand,German Restaurant,Gym Pool,Hardware Store,Hawaiian Restaurant,Heliport,Himalayan Restaurant,Hostel,Hot Dog Joint,Hotel Bar,Hotpot Restaurant,Indie Theater,Irish Pub,Israeli Restaurant,Japanese Curry Restaurant,Jewish Restaurant,Karaoke Bar,Kids Store,

In [120]:
# set number of clusters
kclusters = 5

neigh_grouped_clustering = neigh_grouped.drop('Neighborhood', 1).fillna(0)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neigh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 4, 0, 4, 4, 0, 0, 0])

In [121]:
df_tor4=df_tor3.drop(['Postal Code'],axis=1)
df_tor4['City']='Toronto'
df_man=manhattan_data
df_man['City']='New York'
neigh_merged=pd.merge(df_tor4, df_man, how='outer').reset_index(drop=True)

In [122]:
# add clustering labels
neigh_merged['Cluster Labels'] = kmeans.labels_
#merge the data
final=pd.merge(left=neigh_merged, right=neigh_df ,on=['Neighborhood'])

Let's visualize the final result in the following table: 

In [123]:
final

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,Toronto,4,Trail,Pub,Health Food Store,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Toronto,4,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,Toronto,4,Park,Fish & Chips Shop,Sandwich Place,Light Rail Station,Italian Restaurant,Burrito Place,Liquor Store,Restaurant,Ice Cream Shop,Pub
3,East Toronto,Studio District,43.659526,-79.340923,Toronto,4,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place,Cheese Shop
4,Central Toronto,Lawrence Park,43.728020,-79.388790,Toronto,0,Park,Swim School,Bus Line,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
5,Central Toronto,Davisville North,43.712751,-79.390197,Toronto,4,Park,Hotel,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Department Store,Food & Drink Shop,Doner Restaurant,Dog Run,Distribution Center
6,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,Toronto,4,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Salon / Barbershop,Spa,Restaurant,Sporting Goods Shop,Fast Food Restaurant
7,Central Toronto,Davisville,43.704324,-79.388790,Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Gym,Brewery,Diner
8,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,Toronto,0,Restaurant,Park,Trail,Tennis Court,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
9,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,Toronto,0,Pub,Coffee Shop,Bagel Shop,Supermarket,Sports Bar,Bank,Restaurant,Pizza Place,Liquor Store,Sushi Restaurant


## 5. Results: Visualizing the resulting Clusters

We use the matplotlib and folium packages to visualize the clusters on a map of both cities

In [132]:
n1=neigh_merged.loc[neigh_merged['City']=='Toronto']
n2=neigh_merged.loc[neigh_merged['City']=='New York']

In [141]:
# create map
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters1 = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(n1['Latitude'], n1['Longitude'], 
                                  n1['Neighborhood'], 
                                  n1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1

In [142]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(n2['Latitude'], n2['Longitude'], 
                                  n2['Neighborhood'], 
                                  n2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

## 6. Discussion

The aim of this analysis was to find out similar neighborhoods for a person relocating in New York city.
The maps above show us that if your friend want to move from a neighborhood in Toronto to a neighborhood in Manhattan, he has to choose the neighborhood with the same color displayed if he want to find the same kind of venues of his living zone. Consequently he’s lucky if he live in an orange or red zone of the map above in Toronto.

## 7. Conclusion

The scope of the analysis has been achieved. However the model created can easily be replicated again and again with data from other cities by using the Foursquare API. This show us the potentiality of Data Science in real life problems